# Caffees in Kaunas

# Data

**A description of the data**: the data used to solve this problem is geolocation data collected from [FourSquare](https://foursquare.com/). Adequate explanation and discussion, with examples, of the data is the following. Data is a single dataframe, containing at least a location of the café. **Explanation** of the location data is a standard tuple `(lat, lng)`, where `lat` stands for latitude and `lng` for longitude. Some other metadata like name, postal code and so on is also collected, but let us **discuss** that they are not absolutely necessary for the analysis. **Example** of the data:
        
        
   | identifier               | Name             | Shortname    |   Latitude | Londitude |
   | ------------------------ | ---------------- | ------------ |  --------- | --------- |
   | 55576128498eb6b111d5b64a | Holy Donut       | Donuts       |  54.897056 | 23.898103 |
   | 57ed3c5f498e303700b3f5b5 | Green Cafe       | Coffee       |  54.897564 | 23.910222 |
   | 4c9dd3aaca44236a3c722499 | Spurginė         | Donuts       |  54.897560 | 23.908816 |
   | 5381a0cd498e77bc15f3408d | GREEN cafe       | Coffee       |  54.897224 | 23.916596 |
   | 4cffd08321ea6ea8b5bb3e9f | CH Chocolaterie  | Desserts     |  54.897215 | 23.886822 |


## Analysis

In [1]:
import pandas as pd
import requests

In [2]:
!conda install -c conda-forge folium=0.5.0 --yes

import folium # plotting library

print('Folium installed')
print('Libraries imported.')

Solving environment: done

# All requested packages already installed.

Folium installed
Libraries imported.


In [3]:
# The code was removed by Watson Studio for sharing.

First a GET request for FourSquare API is constructed. Here, we find coffee near Kaunas center, in 2000 m radius. In general it's better to let requests to handle adding parameters to url than manually construct it.

In [4]:
request_parameters = {
    "client_id": CLIENT_ID,
    "client_secret": CLIENT_SECRET,
    "v": '20180605',
    "section": "coffee",
    "near": "Kaunas",
    "radius": 2000,
    "limit": 50}

data = requests.get("https://api.foursquare.com/v2/venues/explore", params=request_parameters)

Parsing response to JSON. We get to following keys in our dictionary:

In [5]:
d = data.json()["response"]
d.keys()

dict_keys(['suggestedFilters', 'geocode', 'headerLocation', 'headerFullLocation', 'headerLocationGranularity', 'query', 'totalResults', 'suggestedBounds', 'groups'])

Most of the items on dictionary are giving some general information like the name of the city, it's geo code and so on. groups contains the actual search result.

In [6]:
d["headerLocationGranularity"], d["headerLocation"], d["headerFullLocation"]

('city', 'Kaunas', 'Kaunas')

In [7]:
d["suggestedBounds"], d["totalResults"]

({'ne': {'lat': 54.90575864257144, 'lng': 23.92891068879652},
  'sw': {'lat': 54.889970892710664, 'lng': 23.884818034182615}},
 40)

In [8]:
d["geocode"]

{'what': '',
 'where': 'kaunas',
 'center': {'lat': 54.9, 'lng': 23.9},
 'displayString': 'Kaunas, Lithuania',
 'cc': 'LT',
 'geometry': {'bounds': {'ne': {'lat': 54.97540055932882,
    'lng': 24.084777000000088},
   'sw': {'lat': 54.82487520303437, 'lng': 23.790868691684864}}},
 'slug': 'kaunas-lithuania',
 'longId': '72057594038526252'}

In groups we again have several different items, where type and name are irrelevant for given task. The full path to items is data.json()["response"]["groups"][0]["items"].

Let's print couple of first items to get again better idea of the structure of the data:

In [9]:
d["groups"][0].keys()

dict_keys(['type', 'name', 'items'])

In [10]:
d["groups"][0]["type"], d["groups"][0]["name"]

('Recommended Places', 'recommended')

In [11]:
items = d["groups"][0]["items"]
print("number of items: %i" % len(items))
items[0]

number of items: 40


{'reasons': {'count': 0,
  'items': [{'summary': 'This spot is popular',
    'type': 'general',
    'reasonName': 'globalInteractionReason'}]},
 'venue': {'id': '55576128498eb6b111d5b64a',
  'name': 'Holy Donut',
  'location': {'address': 'Vilniaus g. 37',
   'lat': 54.89705613716957,
   'lng': 23.89810324154634,
   'labeledLatLngs': [{'label': 'display',
     'lat': 54.89705613716957,
     'lng': 23.89810324154634}],
   'cc': 'LT',
   'city': 'Kaunas',
   'state': 'Kauno apskritis',
   'country': 'Lietuva',
   'formattedAddress': ['Vilniaus g. 37', 'Kaunas', 'Lietuva']},
  'categories': [{'id': '4bf58dd8d48988d148941735',
    'name': 'Donut Shop',
    'pluralName': 'Donut Shops',
    'shortName': 'Donuts',
    'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/donuts_',
     'suffix': '.png'},
    'primary': True}],
  'photos': {'count': 0, 'groups': []}},
 'referralId': 'e-5-55576128498eb6b111d5b64a-0'}

In [12]:
items[1]

{'reasons': {'count': 0,
  'items': [{'summary': 'This spot is popular',
    'type': 'general',
    'reasonName': 'globalInteractionReason'}]},
 'venue': {'id': '57ed3c5f498e303700b3f5b5',
  'name': 'Green Cafe',
  'location': {'address': 'Laisvės alėja 80',
   'lat': 54.89756352079816,
   'lng': 23.91022244171129,
   'labeledLatLngs': [{'label': 'display',
     'lat': 54.89756352079816,
     'lng': 23.91022244171129}],
   'postalCode': '44249',
   'cc': 'LT',
   'city': 'Kaunas',
   'state': 'Kauno apskritis',
   'country': 'Lietuva',
   'formattedAddress': ['Laisvės alėja 80', '44249 Kaunas', 'Lietuva']},
  'categories': [{'id': '4bf58dd8d48988d1e0931735',
    'name': 'Coffee Shop',
    'pluralName': 'Coffee Shops',
    'shortName': 'Coffee Shop',
    'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/coffeeshop_',
     'suffix': '.png'},
    'primary': True}],
  'photos': {'count': 0, 'groups': []}},
 'referralId': 'e-5-57ed3c5f498e303700b3f5b5-1'}

Based on this, a simple for loop over the items is collecting some relevant data from the array, and after that DataFrame is created based on this parsed data.

In [13]:
df_raw = []
for item in items:
    venue = item["venue"]
    categories, uid, name, location = venue["categories"], venue["id"], venue["name"], venue["location"]
    assert len(categories) == 1
    shortname = categories[0]["shortName"]
    lat = location["lat"]
    lng = location["lng"]
    datarow = (uid, name, shortname, lat, lng)
    df_raw.append(datarow)
df = pd.DataFrame(df_raw, columns=["uid", "name", "shortname", "lat", "lng"])
print("found %i cafes" % len(df))
df.head()

found 40 cafes


,uid,name,shortname,lat,lng
0,55576128498eb6b111d5b64a,Holy Donut,Donuts,54.897056,23.898103
1,57ed3c5f498e303700b3f5b5,Green Cafe,Coffee Shop,54.897564,23.910222
2,4c9dd3aaca44236a3c722499,Spurginė,Donuts,54.897560,23.908816
3,5381a0cd498e77bc15f3408d,GREEN cafe,Coffee Shop,54.897224,23.916596
4,4cffd08321ea6ea8b5bb3e9f,CH Chocolaterie,Desserts,54.897215,23.886822


Some density based estimator is giving a good tip where to start a new coffee business. There's a HeatMap plugin ready in Folium, let's use that, and visualize all the existing Cafes to same map:

In [14]:
kaunas_center = d["geocode"]["center"]
kaunas_center

{'lat': 54.9, 'lng': 23.9}

In [15]:
from folium import plugins

# create map of Helsinki using latitude and longitude values
map_kaunas = folium.Map(location=[kaunas_center["lat"], kaunas_center["lng"]], zoom_start=14)

def add_markers(df):
    for (j, row) in df.iterrows():
        label = folium.Popup(row["name"], parse_html=True)
        folium.CircleMarker(
            [row["lat"], row["lng"]],
            radius=5,
            popup=label,
            color='blue',
            fill=True,
            fill_color='#3186cc',
            fill_opacity=0.7,
            parse_html=False).add_to(map_kaunas)

add_markers(df)
hm_data = df[["lat", "lng"]].as_matrix().tolist()
map_kaunas.add_child(plugins.HeatMap(hm_data))

map_kaunas

/opt/conda/envs/Python36/lib/python3.6/site-packages/ipykernel/__main__.py:20: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.


## Results


Based on these results, one possibly good location for new Cafe would be in Laisves aleja:

In [16]:
lat = 54.897560
lng = 23.908816
map_kaunas = folium.Map(location=[lat, lng], zoom_start=17)
add_markers(df)
folium.CircleMarker(
    [lat, lng],
    radius=15,
    popup="Our Cafe!",
    color='red',
    fill=True,
    fill_color='#3186cc',
    fill_opacity=0.7,
    parse_html=False).add_to(map_kaunas)
map_kaunas